In [1]:
pip install -U langgraph langchain langchain-community duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.8/151.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.8/442.8 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 56.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 9.8 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.66
    Uninstalling langchain-core-0.3.66:
      Successfully uninstalled langchain-core-0.3.66
  Attemptin

In [2]:
pip install -qU "langchain[google-genai]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 19.3 MB/s eta 0:00:0000:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model

model= init_chat_model("gemini-2.0-flash", model_provider="google_genai")

Enter API key for Google Gemini:  ········


In [4]:
from langgraph.graph import START, StateGraph  # To initialize Langgraph workflow
from langgraph.checkpoint.memory import MemorySaver # To log the state chnages form each node
from langchain_core.messages import HumanMessage, AIMessage # To enable to chat woth the agent
from langchain_core.tools import tool # To use various tools
from langchain_community.tools import DuckDuckGoSearchRun # Tool for searching
from typing import TypedDict, Annotated, Sequence # Used to create custate state
from langgraph.graph.message import add_messages # meta fucntion to add messages in state
from typing_extensions import NotRequired

In [5]:
class AgentState(TypedDict):
    # Stores only Human and AI message in this format
    messages: Annotated[Sequence[HumanMessage | AIMessage], add_messages]
    
    # Used to route the flow to tools, NotRequired is used because not always you would need tools
    route: NotRequired[str] 

In [6]:
@tool
def calculator(expression: str)-> str:
    """Evaluates a mathematical expression"""
    try:
        return str(eval(expression))
    except:
        return "Invalid expression"

In [7]:
search_tool= DuckDuckGoSearchRun()

def search(query: str)-> str:
    return search_tool.run(query)

In [8]:
def router(state: AgentState):
    messages = state["messages"]
    # Only look at the last human message
    for msg in reversed(messages):
        if isinstance(msg, HumanMessage):
            last_input = msg.content.lower()
            break
    else:
        return {"route": "chat"}  # fallback

    if "calculate" in last_input or any(op in last_input for op in "+-*/%"):
        return {"route": "calculator"}
    elif "search" in last_input or "who is" in last_input or "what is" in last_input:
        return {"route":"search"}
    else:
        return {"route": "chat"}


In [9]:
def call_model(state: AgentState):
    response = model.invoke(state["messages"])
    return {
        "messages": [*state["messages"], AIMessage(content=response.content)],
        "route": None  
    }

def use_calculator(state: AgentState):
    question = state["messages"][-1].content
    result = calculator.invoke(question)
    return {
        "messages": [*state["messages"], AIMessage(content=result)],
        "route": None  
    }

def use_search(state: AgentState):
    question = state["messages"][-1].content
    result = search(question)
    return {
        "messages": [*state["messages"], AIMessage(content=result)],
        "route": None  
    }


1. "router"
This is the node you're connecting from. It will run first.

2. lambda x: x["route"]
This function is called with the state (which is a dictionary), and it returns state["route"].


So whatever value the router node saves into state["route"] (like "chat" or "search"), this lambda pulls it out.


In [10]:
workflow= StateGraph(AgentState)

workflow.add_node("router", router)
workflow.add_node("chat", call_model)
workflow.add_node("search", use_search)
workflow.add_node("calculator", use_calculator)

workflow.set_entry_point("router")

workflow.add_conditional_edges(
    "router",
    lambda x: x["route"],
    {
        "chat": "chat",
        "calculator": "calculator",
        "search": "search",
    }
)

workflow.add_edge("chat", "router")
workflow.add_edge("calculator", "router")
workflow.add_edge("search", "router")

memory= MemorySaver()
app= workflow.compile(checkpointer=memory)

In [11]:
# This is usded to storedata about differnt sessions, used when differnt users are there
config = {"configurable": {"thread_id": "abc13"}} 

In [12]:
state = {"messages": []}

while True:
    query = input("You: ")
    if query == "N-":
        break
    state["messages"].append(HumanMessage(content=query))
    state = app.invoke(state, config)
    print("Bot:", state["messages"][-1].content)

You:  Hi


GraphRecursionError: Recursion limit of 25 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/GRAPH_RECURSION_LIMIT